In [1]:
# import statements
import pandas as pd
import numpy as np
from io import StringIO
# from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
# display settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# Read in our beautiful TN non-spatial dataset
tn_data = pd.read_csv('../data/TN_aggregated_data.csv', index_col=[0])
tn_data.head()

,county,pct_children_under_18,pct_people_living_in_rural_areas,pct_people_of_color,pct_people_with_disabilities,pct_senior_citizens,total_housing_units_2019,county_full,population_est,pct_insured,race_afam,race_asam,race_latn,race_natam,race_paci,race_white,race_othr,race_multi,disability_y,disability_n,edu_lessHS,edu_HSgrad,edu_assoc,edu_min_bac,employed_est,unemployed_est,tot_w_poverty_lvl_data,tot_poverty_less100,tot_poverty_101_138,tot_poverty_138_399,tot_poverty_400_up,area_miles2,socioeconomic,household_comp_and_disability,minority_status_and_language,housing_type_and_transportation,total_vulnerability,pct_uninsured,gdp_2018,gdp_per_cap_2018,mag_mean,mag_median,mag_max,depth_mean,depth_median,depth_min,num_earthquakes,pct_prob_100y_median,pct_prob_100y_max
0,anderson,21.9,34.66,6.8,18.6,17.4,35143,"Anderson County, Tennessee",74862,90.8,2365,1018,2094,377,29,66911,637,2021,14157,60705,6828,17453,15622,12480,30244,1969,74292,12022,5901,33240,23129,337.203175,0.2128,0.6277,0.6596,0.6489,0.4043,9.2,4770850.0,63.728594,1.820000,1.725,2.23,15.995000,15.185,12.98,4.0,16.68,23.48
1,bedford,26.8,55.60,16.0,15.7,12.9,19531,"Bedford County, Tennessee",47184,88.7,3384,79,5682,27,29,36541,2404,1495,7526,39658,5814,12314,7346,4999,19883,940,46923,7420,4105,24435,10963,473.638959,0.3511,0.6915,0.9681,0.5319,0.6809,11.3,1824402.0,38.665692,2.770000,2.770,2.77,6.520000,6.520,6.52,1.0,16.47,19.56
2,benton,20.5,78.46,3.9,24.6,19.9,9172,"Benton County, Tennessee",15960,90.3,507,34,274,69,0,14835,0,285,4297,11663,2161,5447,2519,1404,5185,443,15922,3403,2114,7145,3260,394.306039,0.8085,0.7766,0.2553,0.6277,0.7766,9.7,386699.0,24.229261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.18,32.83
3,bledsoe,21.5,100.00,6.4,30.0,15.4,5796,"Bledsoe County, Tennessee",13436,87.0,347,8,252,223,0,12220,23,371,3603,9833,2489,3692,2362,1390,5367,527,13391,2721,1342,6438,2890,406.424786,0.8723,0.1809,0.8085,0.9681,0.9574,13.0,224732.0,16.726109,1.656667,1.650,2.50,17.120556,17.105,4.38,18.0,18.31,24.48
4,blount,22.3,32.63,5.6,14.8,16.1,58942,"Blount County, Tennessee",127292,91.2,3328,993,4145,369,127,116154,259,2481,18304,108988,9548,30868,26997,21624,54149,2857,126101,14808,8975,59213,43105,558.816658,0.0426,0.0426,0.5426,0.1702,0.0745,8.8,5384691.0,42.301881,1.918889,1.920,4.70,12.531778,13.060,0.04,135.0,23.80,30.84


In [4]:
tn_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 0 to 94
Data columns (total 49 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   county                            95 non-null     object 
 1   pct_children_under_18             95 non-null     float64
 2   pct_people_living_in_rural_areas  95 non-null     float64
 3   pct_people_of_color               95 non-null     float64
 4   pct_people_with_disabilities      95 non-null     float64
 5   pct_senior_citizens               95 non-null     float64
 6   total_housing_units_2019          95 non-null     int64  
 7   county_full                       95 non-null     object 
 8   population_est                    95 non-null     int64  
 9   pct_insured                       95 non-null     float64
 10  race_afam                         95 non-null     int64  
 11  race_asam                         95 non-null     int64  
 12  race_latn 

## Step 1. Identify numeric variables of interest that need to be "reverse-coded"  

LOWER NUMBERS = LOWER RISK/VULNERABILITY
HIGHER NUMBERS = HIGHER RISK/VULNERABILITY  

#### Variables that need to be created:  
tn_data['pop_density'] = tn_data['population_est'] / tn_data['area_miles2']   

tn_data['pct_edu_HSgrad'] = tn_data['edu_HSgrad'] / tn_data['population_est']   **percent w/edu above less than HS**  
tn_data['pct_edu_assoc'] = tn_data['edu_assoc'] / tn_data['population_est']  
tn_data['pct_edu_min_bac'] = tn_data['edu_min_bac'] / tn_data['population_est']  

**calculate the percentile rank for remaining edu variables; .rank(pct = True) ranks each county on a scale from 0 (lowest %) to 1 (highest %); after creating, these need to be flipped for analyses so that high values = high vulnerability**  
tn_data['pctrnk_edu_HSgrad'] = tn_data.pct_edu_HSgrad.rank(pct = True)    
tn_data['pctrnk_edu_assoc'] = tn_data.pct_edu_assoc.rank(pct = True)      
tn_data['pctrnk_edu_min_bac'] = tn_data.pct_edu_min_bac.rank(pct = True)    

**perform the same function as above for poverty bands using the variables below**  
tot_w_poverty_lvl_data  
tot_poverty_138_399  
tot_poverty_400_up 

#### Variables that need to be retrieved:


#### Variables that need to be "reverse-coded" (made negative):  
tn_data['pctrnk_edu_HSgrad']   
tn_data['pctrnk_edu_assoc']   
tn_data['pctrnk_edu_min_bac']   
tot_poverty_138_399  (as pct rank)  
tot_poverty_400_up  (as pct rank)  
gdp_2018  **(higher values should mean lower risk)**  
gdp_per_cap_2018  
depth_mean  **(lower values should mean higher risk)**  
depth_median  
depth_min

#### Variables going in the correct direction:  
**[higher number = higher risk]**  
total_housing_units_2019   
population_est    
socioeconomic  
household_comp_and_disability  
minority_status_and_language  
housing_type_and_transportation  
total_vulnerability  
pct_uninsured  
mag_mean  
mag_median  
mag_max  
num_earthquakes  
pct_prob_100y_median  
pct_prob_100y_max  

**[lower number = lower risk]**
we don't have any variables that fall into this category

#### Variables that will not be used in risk calculation equation (because they are already part of SVI or other components):  
*pct_children_under_18  
pct_people_of_color    
pct_people_with_disabilities    
pct_senior_citizens  
pct_insured    **(use pct_uninsured instead)**  
race_afam  
race_asam  
race_latn  
race_natam  
race_paci  
race_white  
race_othr  
race_multi  
disability_y  
disability_n 
edu_lessHS  
edu_HSgrad  
edu_assoc  
edu_min_bac  
employed_est  
unemployed_est  
pct_people_living_in_rural_areas    **(leaving off because population density will be more useful for risk calc)**  
tot_poverty_less100    **(already present in SES data)**  
tot_poverty_101_138  

In [5]:
# create variables that need to be created
tn_data['pop_density'] = tn_data['population_est'] / tn_data['area_miles2']
tn_data['pctrnk_pop_density'] = tn_data.pop_density.rank(pct=True)

In [6]:
# # get percent rank in default order so that higher percentages correspond with higher risk rankings
# tn_data['pctrnk_edu_HSgrad'] = tn_data.pct_edu_HSgrad.rank(pct = True)    
# tn_data['pctrnk_edu_assoc'] = tn_data.pct_edu_assoc.rank(pct = True)      
# tn_data['pctrnk_edu_min_bac'] = tn_data.pct_edu_min_bac.rank(pct = True)

In [7]:
# # check output
# tn_data.head()

In [8]:
# get percent rank in reverse so that higher percentages correspond with lower risk rankings
tn_data['pct_edu_HSgrad'] = tn_data['edu_HSgrad'] / tn_data['population_est']   
tn_data['pctrnk_edu_HSgrad'] = tn_data.pct_edu_HSgrad.rank(pct = True, ascending = False)

tn_data['pct_edu_assoc'] = tn_data['edu_assoc'] / tn_data['population_est']
tn_data['pctrnk_edu_assoc'] = tn_data.pct_edu_assoc.rank(pct = True, ascending = False)

tn_data['pct_edu_min_bac'] = tn_data['edu_min_bac'] / tn_data['population_est']
tn_data['pctrnk_edu_min_bac'] = tn_data.pct_edu_min_bac.rank(pct = True, ascending = False)

In [9]:
# check output
tn_data.head()

,county,pct_children_under_18,pct_people_living_in_rural_areas,pct_people_of_color,pct_people_with_disabilities,pct_senior_citizens,total_housing_units_2019,county_full,population_est,pct_insured,race_afam,race_asam,race_latn,race_natam,race_paci,race_white,race_othr,race_multi,disability_y,disability_n,edu_lessHS,edu_HSgrad,edu_assoc,edu_min_bac,employed_est,unemployed_est,tot_w_poverty_lvl_data,tot_poverty_less100,tot_poverty_101_138,tot_poverty_138_399,tot_poverty_400_up,area_miles2,socioeconomic,household_comp_and_disability,minority_status_and_language,housing_type_and_transportation,total_vulnerability,pct_uninsured,gdp_2018,gdp_per_cap_2018,mag_mean,mag_median,mag_max,depth_mean,depth_median,depth_min,num_earthquakes,pct_prob_100y_median,pct_prob_100y_max,pop_density,pctrnk_pop_density,pct_edu_HSgrad,pctrnk_edu_HSgrad,pct_edu_assoc,pctrnk_edu_assoc,pct_edu_min_bac,pctrnk_edu_min_bac
0,anderson,21.9,34.66,6.8,18.6,17.4,35143,"Anderson County, Tennessee",74862,90.8,2365,1018,2094,377,29,66911,637,2021,14157,60705,6828,17453,15622,12480,30244,1969,74292,12022,5901,33240,23129,337.203175,0.2128,0.6277,0.6596,0.6489,0.4043,9.2,4770850.0,63.728594,1.820000,1.725,2.23,15.995000,15.185,12.98,4.0,16.68,23.48,222.008586,0.842105,0.233136,0.852632,0.208677,0.115789,0.166707,0.136842
1,bedford,26.8,55.60,16.0,15.7,12.9,19531,"Bedford County, Tennessee",47184,88.7,3384,79,5682,27,29,36541,2404,1495,7526,39658,5814,12314,7346,4999,19883,940,46923,7420,4105,24435,10963,473.638959,0.3511,0.6915,0.9681,0.5319,0.6809,11.3,1824402.0,38.665692,2.770000,2.770,2.77,6.520000,6.520,6.52,1.0,16.47,19.56,99.620184,0.663158,0.260978,0.673684,0.155688,0.821053,0.105947,0.463158
2,benton,20.5,78.46,3.9,24.6,19.9,9172,"Benton County, Tennessee",15960,90.3,507,34,274,69,0,14835,0,285,4297,11663,2161,5447,2519,1404,5185,443,15922,3403,2114,7145,3260,394.306039,0.8085,0.7766,0.2553,0.6277,0.7766,9.7,386699.0,24.229261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.18,32.83,40.476174,0.221053,0.341291,0.042105,0.157832,0.800000,0.087970,0.736842
3,bledsoe,21.5,100.00,6.4,30.0,15.4,5796,"Bledsoe County, Tennessee",13436,87.0,347,8,252,223,0,12220,23,371,3603,9833,2489,3692,2362,1390,5367,527,13391,2721,1342,6438,2890,406.424786,0.8723,0.1809,0.8085,0.9681,0.9574,13.0,224732.0,16.726109,1.656667,1.650,2.50,17.120556,17.105,4.38,18.0,18.31,24.48,33.059007,0.115789,0.274784,0.578947,0.175796,0.547368,0.103453,0.515789
4,blount,22.3,32.63,5.6,14.8,16.1,58942,"Blount County, Tennessee",127292,91.2,3328,993,4145,369,127,116154,259,2481,18304,108988,9548,30868,26997,21624,54149,2857,126101,14808,8975,59213,43105,558.816658,0.0426,0.0426,0.5426,0.1702,0.0745,8.8,5384691.0,42.301881,1.918889,1.920,4.70,12.531778,13.060,0.04,135.0,23.80,30.84,227.788485,0.863158,0.242498,0.800000,0.212087,0.084211,0.169877,0.115789


In [10]:
# get percent rank in reverse so that higher percentages correspond with lower risk rankings
tn_data['pct_poverty_138_399'] = tn_data['tot_poverty_138_399'] / tn_data['tot_w_poverty_lvl_data']
tn_data['pctrnk_poverty_138_399'] = tn_data.pct_poverty_138_399.rank(pct = True, ascending = False)

tn_data['pct_poverty_400_up'] = tn_data['tot_poverty_400_up'] / tn_data['tot_w_poverty_lvl_data']
tn_data['pctrnk_poverty_400_up'] = tn_data.pct_poverty_400_up.rank(pct = True, ascending = False)

In [11]:
# get percent rank in reverse so that higher values correspond with lower risk rankings
tn_data['pctrnk_gdp_2018'] = tn_data.gdp_2018.rank(pct = True, ascending = False)
tn_data['pctrnk_gdp_per_cap_2018'] = tn_data.gdp_per_cap_2018.rank(pct = True, ascending = False)

tn_data['pctrnk_depth_mean'] = tn_data.depth_mean.rank(pct = True, ascending = False)
tn_data['pctrnk_depth_median'] = tn_data.depth_median.rank(pct = True, ascending = False)    
tn_data['pctrnk_depth_min']  = tn_data.depth_min.rank(pct = True, ascending = False)

In [12]:
# get percentile rank for variables going in the correct direction (i.e., higher values correspond with higher risk rankings)
tn_data['pctrnk_housing_units'] = tn_data.total_housing_units_2019.rank(pct=True)
tn_data['pctrnk_pop_est'] = tn_data.population_est.rank(pct=True)
tn_data['pctrnk_uninsured'] = tn_data.pct_uninsured.rank(pct=True)
tn_data['pctrnk_mag_mean'] = tn_data.mag_mean.rank(pct=True)
tn_data['pctrnk_mag_median'] = tn_data.mag_median.rank(pct=True)
tn_data['pctrnk_mag_max'] = tn_data.mag_max.rank(pct=True)
tn_data['pctrnk_earthquakes'] = tn_data.num_earthquakes.rank(pct=True)
tn_data['pctrnk_prob_100y_median'] = tn_data.pct_prob_100y_median.rank(pct=True)
tn_data['pctrnk_prob_100y_max'] =  tn_data.pct_prob_100y_max.rank(pct=True)

Low numbers = low risk  
High numbers = high risk  

For fully aggregated data set (prioritize - #1):  
* for every variable, does it follow the right pattern/direction?
* reverse-coded: depth, GDP, GDP per capita (as neg)
* depth (median and max)
* magnitude (median and max)

Generate histograms for each variable of interest; if skewed (probably pop, magnitude), create new column with np.log() to normalize it (prioritize - #3)

Scikitlearn (for weighting) (prioritize - #2)

Calculations
* Cum H, Ex, V
   * Average of each category (weighing every variable equally, regardless of how many components)
* Cum H, Ex, V
   * Sum of each category (biases towards areas with more data points)
(Either way should go in the same direction)

Plug in both cum calc into risk formulas for two columns
risk_avg (also w/np.log() columns)
risk_sum (also w/np.log() columns)

Map risk_avg and risk_sum; throw into widget 

If not informative, do we want to double weights? Play around with formula to get a cleaner visualization to make a case for opportunities to invest

## Step 2 - Calculate Values for Risk Categories  
### NOTE - Just refer to this for the general idea...I found errors in the code, so I made changes in the cells that follow  
  
EmR / DR / CatR = H + Ex × V    
  
#### Code for calculating hazzard score column:    
tn_data[‘hazzard_wcs’] = tn_data[‘pctrnk_earthquakes’] + tn_data[‘pctrnk_mag_max’] + tn_data[‘pctrnk_prob_100y_max’] + tn_data[‘pctrnk_depth_min’]
  
#### Code for calculating exposure score column:  
tn_data[‘exposure_wcs’] = tn_data[‘pctrnk_pop_est’] + tn_data[‘pctrnk_gdp_2018’] + tn_data[‘pctrnk_gdp_per_cap_2018’] + tn_data[‘pctrnk_housing_units’]

(add tn_data['pctrnk_pop_density'] to this category)  
  
#### Code for calculating vulnerability score column:  
tn_data[‘vulnerability_wcs’] = tn_data[‘socioeconomic’] + tn_data[‘household_comp_and_disability’] + tn_data[‘minority_status_and_language’] + tn_data[‘housing_type_and_transportation’] + tn_data[‘pctrnk_uninsured’]

(add tn_data['pctrnk_edu_min_bac'] to this category)  
  
#### Code for calculating risk score column:  
tn_data[‘risk_calc_wcs’] = tn_data[‘hazzard_wcs’] + tn_data[‘exposure_wcs’] * tn_data[‘vulnerability_wcs’]  
  
The 'wcs' ending refers to 'worst case scenario' since all of these calcuations use the max values for mag, prob_100y, and depth. We could use a different ending/abbreviation for the inclusion of mean and median values if desired.

In [13]:
# using max hazzard variables
# Code for calculating hazzard score column:  
tn_data['hazzard_wcs'] = tn_data['pctrnk_earthquakes'] + tn_data['pctrnk_mag_max'] + tn_data['pctrnk_prob_100y_max'] + tn_data['pctrnk_depth_min']
tn_data['pctrnk_hazzard_wcs'] = tn_data.hazzard_wcs.rank(pct=True)

# Code for calculating exposure score column:
tn_data['exposure_wcs'] = tn_data['pctrnk_pop_est'] + tn_data['pctrnk_pop_density'] + tn_data['pctrnk_gdp_2018'] + tn_data['pctrnk_gdp_per_cap_2018'] + tn_data['pctrnk_housing_units']
tn_data['pctrnk_exposure_wcs'] = tn_data.exposure_wcs.rank(pct=True)

# Code for calculating vulnerability score column (adding tn_data['pctrnk_edu_min_bac']):
tn_data['vulnerability_wcs'] = tn_data['socioeconomic'] + tn_data['pctrnk_edu_min_bac'] + tn_data['household_comp_and_disability'] + tn_data['minority_status_and_language'] + tn_data['housing_type_and_transportation'] + tn_data['pctrnk_uninsured']
tn_data['pctrnk_vul_wcs'] = tn_data.vulnerability_wcs.rank(pct=True)

# Code for calculating risk score column:
tn_data['risk_calc_wcs'] = tn_data['pctrnk_hazzard_wcs'] + tn_data['pctrnk_exposure_wcs'] * tn_data['pctrnk_vul_wcs']
tn_data['risk_calc_wcs'] = tn_data.risk_calc_wcs.rank(pct=True)

In [14]:
tn_data

,county,pct_children_under_18,pct_people_living_in_rural_areas,pct_people_of_color,pct_people_with_disabilities,pct_senior_citizens,total_housing_units_2019,county_full,population_est,pct_insured,race_afam,race_asam,race_latn,race_natam,race_paci,race_white,race_othr,race_multi,disability_y,disability_n,edu_lessHS,edu_HSgrad,edu_assoc,edu_min_bac,employed_est,unemployed_est,tot_w_poverty_lvl_data,tot_poverty_less100,tot_poverty_101_138,tot_poverty_138_399,tot_poverty_400_up,area_miles2,socioeconomic,household_comp_and_disability,minority_status_and_language,housing_type_and_transportation,total_vulnerability,pct_uninsured,gdp_2018,gdp_per_cap_2018,mag_mean,mag_median,mag_max,depth_mean,depth_median,depth_min,num_earthquakes,pct_prob_100y_median,pct_prob_100y_max,pop_density,pctrnk_pop_density,pct_edu_HSgrad,pctrnk_edu_HSgrad,pct_edu_assoc,pctrnk_edu_assoc,pct_edu_min_bac,pctrnk_edu_min_bac,pct_poverty_138_399,pctrnk_poverty_138_399,pct_poverty_400_up,pctrnk_poverty_400_up,pctrnk_gdp_2018,pctrnk_gdp_per_cap_2018,pctrnk_depth_mean,pctrnk_depth_median,pctrnk_depth_min,pctrnk_housing_units,pctrnk_pop_est,pctrnk_uninsured,pctrnk_mag_mean,pctrnk_mag_median,pctrnk_mag_max,pctrnk_earthquakes,pctrnk_prob_100y_median,pctrnk_prob_100y_max,hazzard_wcs,pctrnk_hazzard_wcs,exposure_wcs,pctrnk_exposure_wcs,vulnerability_wcs,pctrnk_vul_wcs,risk_calc_wcs
0,anderson,21.9,34.66,6.8,18.6,17.4,35143,"Anderson County, Tennessee",74862,90.8,2365,1018,2094,377,29,66911,637,2021,14157,60705,6828,17453,15622,12480,30244,1969,74292,12022,5901,33240,23129,337.203175,0.2128,0.6277,0.6596,0.6489,0.4043,9.2,4770850.0,63.728594,1.820000,1.725,2.23,15.995000,15.185,12.98,4.0,16.680,23.48,222.008586,0.842105,0.233136,0.852632,0.208677,0.115789,0.166707,0.136842,0.447424,0.884211,0.311326,0.189474,0.157895,0.073684,0.149254,0.194030,0.059701,0.821053,0.821053,0.294737,0.208955,0.164179,0.156716,0.529851,0.400000,0.536842,1.283111,0.223881,2.715789,0.621053,2.580579,0.336842,0.223881
1,bedford,26.8,55.60,16.0,15.7,12.9,19531,"Bedford County, Tennessee",47184,88.7,3384,79,5682,27,29,36541,2404,1495,7526,39658,5814,12314,7346,4999,19883,940,46923,7420,4105,24435,10963,473.638959,0.3511,0.6915,0.9681,0.5319,0.6809,11.3,1824402.0,38.665692,2.770000,2.770,2.77,6.520000,6.520,6.52,1.0,16.470,19.56,99.620184,0.663158,0.260978,0.673684,0.155688,0.821053,0.105947,0.463158,0.520747,0.084211,0.233638,0.526316,0.294737,0.326316,0.716418,0.686567,0.223881,0.631579,0.663158,0.763158,0.970149,0.970149,0.552239,0.089552,0.378947,0.242105,1.107777,0.104478,2.578947,0.505263,3.768916,0.747368,0.268657
2,benton,20.5,78.46,3.9,24.6,19.9,9172,"Benton County, Tennessee",15960,90.3,507,34,274,69,0,14835,0,285,4297,11663,2161,5447,2519,1404,5185,443,15922,3403,2114,7145,3260,394.306039,0.8085,0.7766,0.2553,0.6277,0.7766,9.7,386699.0,24.229261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.180,32.83,40.476174,0.221053,0.341291,0.042105,0.157832,0.800000,0.087970,0.736842,0.448750,0.873684,0.204748,0.736842,0.768421,0.778947,NaN,NaN,NaN,0.305263,0.231579,0.452632,NaN,NaN,NaN,NaN,0.842105,0.810526,NaN,NaN,2.305263,0.336842,3.657574,0.705263,NaN
3,bledsoe,21.5,100.00,6.4,30.0,15.4,5796,"Bledsoe County, Tennessee",13436,87.0,347,8,252,223,0,12220,23,371,3603,9833,2489,3692,2362,1390,5367,527,13391,2721,1342,6438,2890,406.424786,0.8723,0.1809,0.8085,0.9681,0.9574,13.0,224732.0,16.726109,1.656667,1.650,2.50,17.120556,17.105,4.38,18.0,18.310,24.48,33.059007,0.115789,0.274784,0.578947,0.175796,0.547368,0.103453,0.515789,0.480771,0.557895,0.215817,0.684211,0.894737,0.978947,0.104478,0.104478,0.440299,0.115789,0.168421,0.936842,0.104478,0.119403,0.388060,0.731343,0.484211,0.578947,2.138649,0.567164,2.273684,0.300000,4.282432,0.915789,0.626866
4,blount,22.3,32.63,5.6,14.8,16.1,58942,"Blount County, Tennessee",127292,91.2,3328,993,4145,369,127,116154,259,2481,18304,108988,9548,30868,26997,21624,54149,2857,126101,14808,8975,59213,43105,558.816658,0.0426,0.0426,0.5426,0.1702,0.0745,8.8,5384691.0,42.301881,1.918889,1.

In [15]:
# using median hazzard variables
# Code for calculating hazzard score column:  
tn_data['hazzard_norm'] = tn_data['pctrnk_earthquakes'] + tn_data['pctrnk_mag_median'] + tn_data['pctrnk_prob_100y_median'] + tn_data['pctrnk_depth_median']
tn_data['pctrnk_hazzard_norm'] = tn_data.hazzard_norm.rank(pct=True)

# Code for calculating exposure score column:
tn_data['exposure_norm'] = tn_data['pctrnk_pop_est'] + tn_data['pctrnk_pop_density'] + tn_data['pctrnk_gdp_2018'] + tn_data['pctrnk_gdp_per_cap_2018'] + tn_data['pctrnk_housing_units']
tn_data['pctrnk_exposure_norm'] = tn_data.exposure_norm.rank(pct=True)

# Code for calculating vulnerability score column (adding tn_data['pctrnk_edu_min_bac']):
tn_data['vulnerability_norm'] = tn_data['socioeconomic'] + tn_data['pctrnk_edu_min_bac'] + tn_data['household_comp_and_disability'] + tn_data['minority_status_and_language'] + tn_data['housing_type_and_transportation'] + tn_data['pctrnk_uninsured']
tn_data['pctrnk_vul_norm'] = tn_data.vulnerability_norm.rank(pct=True)

# Code for calculating risk score column:
tn_data['risk_calc_norm'] = tn_data['pctrnk_hazzard_norm'] + tn_data['pctrnk_exposure_norm'] * tn_data['pctrnk_vul_norm']
tn_data['risk_calc_norm'] = tn_data.risk_calc_norm.rank(pct=True)

In [16]:
tn_data

,county,pct_children_under_18,pct_people_living_in_rural_areas,pct_people_of_color,pct_people_with_disabilities,pct_senior_citizens,total_housing_units_2019,county_full,population_est,pct_insured,race_afam,race_asam,race_latn,race_natam,race_paci,race_white,race_othr,race_multi,disability_y,disability_n,edu_lessHS,edu_HSgrad,edu_assoc,edu_min_bac,employed_est,unemployed_est,tot_w_poverty_lvl_data,tot_poverty_less100,tot_poverty_101_138,tot_poverty_138_399,tot_poverty_400_up,area_miles2,socioeconomic,household_comp_and_disability,minority_status_and_language,housing_type_and_transportation,total_vulnerability,pct_uninsured,gdp_2018,gdp_per_cap_2018,mag_mean,mag_median,mag_max,depth_mean,depth_median,depth_min,num_earthquakes,pct_prob_100y_median,pct_prob_100y_max,pop_density,pctrnk_pop_density,pct_edu_HSgrad,pctrnk_edu_HSgrad,pct_edu_assoc,pctrnk_edu_assoc,pct_edu_min_bac,pctrnk_edu_min_bac,pct_poverty_138_399,pctrnk_poverty_138_399,pct_poverty_400_up,pctrnk_poverty_400_up,pctrnk_gdp_2018,pctrnk_gdp_per_cap_2018,pctrnk_depth_mean,pctrnk_depth_median,pctrnk_depth_min,pctrnk_housing_units,pctrnk_pop_est,pctrnk_uninsured,pctrnk_mag_mean,pctrnk_mag_median,pctrnk_mag_max,pctrnk_earthquakes,pctrnk_prob_100y_median,pctrnk_prob_100y_max,hazzard_wcs,pctrnk_hazzard_wcs,exposure_wcs,pctrnk_exposure_wcs,vulnerability_wcs,pctrnk_vul_wcs,risk_calc_wcs,hazzard_norm,pctrnk_hazzard_norm,exposure_norm,pctrnk_exposure_norm,vulnerability_norm,pctrnk_vul_norm,risk_calc_norm
0,anderson,21.9,34.66,6.8,18.6,17.4,35143,"Anderson County, Tennessee",74862,90.8,2365,1018,2094,377,29,66911,637,2021,14157,60705,6828,17453,15622,12480,30244,1969,74292,12022,5901,33240,23129,337.203175,0.2128,0.6277,0.6596,0.6489,0.4043,9.2,4770850.0,63.728594,1.820000,1.725,2.23,15.995000,15.185,12.98,4.0,16.680,23.48,222.008586,0.842105,0.233136,0.852632,0.208677,0.115789,0.166707,0.136842,0.447424,0.884211,0.311326,0.189474,0.157895,0.073684,0.149254,0.194030,0.059701,0.821053,0.821053,0.294737,0.208955,0.164179,0.156716,0.529851,0.400000,0.536842,1.283111,0.223881,2.715789,0.621053,2.580579,0.336842,0.223881,1.288060,0.074627,2.715789,0.621053,2.580579,0.336842,0.059701
1,bedford,26.8,55.60,16.0,15.7,12.9,19531,"Bedford County, Tennessee",47184,88.7,3384,79,5682,27,29,36541,2404,1495,7526,39658,5814,12314,7346,4999,19883,940,46923,7420,4105,24435,10963,473.638959,0.3511,0.6915,0.9681,0.5319,0.6809,11.3,1824402.0,38.665692,2.770000,2.770,2.77,6.520000,6.520,6.52,1.0,16.470,19.56,99.620184,0.663158,0.260978,0.673684,0.155688,0.821053,0.105947,0.463158,0.520747,0.084211,0.233638,0.526316,0.294737,0.326316,0.716418,0.686567,0.223881,0.631579,0.663158,0.763158,0.970149,0.970149,0.552239,0.089552,0.378947,0.242105,1.107777,0.104478,2.578947,0.505263,3.768916,0.747368,0.268657,2.125216,0.537313,2.578947,0.505263,3.768916,0.747368,0.701493
2,benton,20.5,78.46,3.9,24.6,19.9,9172,"Benton County, Tennessee",15960,90.3,507,34,274,69,0,14835,0,285,4297,11663,2161,5447,2519,1404,5185,443,15922,3403,2114,7145,3260,394.306039,0.8085,0.7766,0.2553,0.6277,0.7766,9.7,386699.0,24.229261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.180,32.83,40.476174,0.221053,0.341291,0.042105,0.157832,0.800000,0.087970,0.736842,0.448750,0.873684,0.204748,0.736842,0.768421,0.778947,NaN,NaN,NaN,0.305263,0.231579,0.452632,NaN,NaN,NaN,NaN,0.842105,0.810526,NaN,NaN,2.305263,0.336842,3.657574,0.705263,NaN,NaN,NaN,2.305263,0.336842,3.657574,0.705263,NaN
3,bledsoe,21.5,100.00,6.4,30.0,15.4,5796,"Bledsoe County, Tennessee",13436,87.0,347,8,252,223,0,12220,23,371,3603,9833,2489,3692,2362,1390,5367,527,13391,2721,1342,6438,2890,406.424786,0.8723,0.1809,0.8085,0.9681,0.9574,13.0,224732.0,16.726109,1.656667,1.650,2.50,17.120556,17.105,4.38,18.0,18.310,24.48,33.059007,0.115789,0.274784,0.578947,0.175796,0.547368,0.103453,0.515789,0.480771,0.557895,0.215817,0.684211,0.894737,0.978947,0.104478,0.104478,0.440299,0.115789,0.168421,0.936842,0.104478,0.119403,0.388060,0.731343,0.484211,0.578947,2.138649,0.567164,2.273684,0.300000,4.282432,0

In [18]:
tn_data.to_csv('../data/tn_data_pctrnk_risk_calc.csv')